In [53]:
import pandas as pd
import glob

In [54]:
import json
import random
with open("Intents.json", "r") as file:
    data = json.load(file)


In [55]:
# Hugging Face - Sentence Transformer


In [56]:
!pip install sentence-transformers

In [57]:
sentences = []
tags = []

In [58]:
for intent in data["intents"]:
  for pattern in intent["patterns"]:
    sentences.append(pattern)
    tags.append(intent["tag"])



In [59]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')



In [60]:
embeddings = model.encode(sentences, show_progress_bar=True)
embeddings = np.array(embeddings)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [61]:
print("Embeddings shape:", embeddings.shape)


Embeddings shape: (39, 384)


In [62]:
!pip install faiss-cpu


In [63]:
import faiss

dimensions = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dimensions)
faiss_index.add(np.array(embeddings))



In [64]:
SIMILARITY_THRESHOLD = 0.5


In [65]:
def chatbot_reply(user_input):
    if user_input.lower() in ["bye", "exit", "quit"]:
        return "Goodbye! Feel free to come back anytime.", True

    user_embedding = model.encode([user_input])
    D, I = faiss_index.search(np.array(user_embedding), k=1)

    best_distance = D[0][0]
    best_index = I[0][0]

    if best_distance > SIMILARITY_THRESHOLD:
        return (
            "I'm sorry, I can't provide information about that topic.\n"
            "You might find more details on these sites:\n"
            "- https://www.google.com\n"
            "- https://www.wikipedia.org\n"
            "- https://chat.openai.com"
        ), False

    matched_tag = tags[best_index]
    for intent in data["intents"]:
        if intent["tag"] == matched_tag:
            return random.choice(intent["responses"]), False


In [67]:
if __name__ == "__main__":
    print("Welcome to the Restaurant InfoBot! (type 'bye' to exit)")
    while True:
        user_input = input("You: ")
        response, should_exit = chatbot_reply(user_input)
        print("Bot:", response)
        if should_exit:
            break

Welcome to the Restaurant InfoBot! (type 'bye' to exit)
You: hye
Bot: Good to see you! How can I help with your dining plans?
You: yes
Bot: I'm sorry, I can't provide information about that topic.
You might find more details on these sites:
- https://www.google.com
- https://www.wikipedia.org
- https://chat.openai.com
You: bye
Bot: Goodbye! Feel free to come back anytime.
